In [ ]:
import os
#os.environ['OMP_NUM_THREADS'] = "6"
#os.environ['OMP_PLACES'] = "{0:6:1}"
#os.environ['KMP_AFFINITY'] = "granularity=fine,compact,1,0"

import sys
import copy
import math
import pickle

sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt
#torch.set_num_threads(6)
import implicitmodules.torch as dm

In [ ]:
# First load the deformed peanuts
peanuts = pickle.load(open("../../data/deformed_ellipses.pkl", 'rb'))
peanuts = [torch.tensor(p[:-1]) for p in peanuts['dataset']]

sigma_noise = 0.05
epsilon = 0.2
peanuts_noisy = []
def small_def(x, alphas):
    K = dm.Kernels.K_xx(x, 0.2)
    return x + epsilon * K @ alphas

# Generate noisy data
for p in peanuts:
    alphas = sigma_noise*torch.randn_like(p)
    K = dm.Kernels.K_xx(p, 0.2)
    peanuts_noisy.append(small_def(small_def(p, alphas), alphas))

template = peanuts[0]
peanuts = peanuts[1:]
print("Number of peanuts:", len(peanuts))

In [ ]:
%matplotlib qt5
plt.subplot(1, 2, 1)
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--')
for p in peanuts:
    plt.plot(p[:, 0].numpy(), p[:, 1].numpy())
plt.axis('equal')

plt.subplot(1, 2, 2)
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--')
for p in peanuts_noisy:
    plt.plot(p[:, 0].numpy(), p[:, 1].numpy())
plt.axis('equal')
    
plt.show()

In [ ]:
# A simple polynomial model of order 1
def pol_order_1(pos, a, b, c):
    return a + b*pos[:, 0] + c*pos[:, 1]

def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

def pol_order_3(pos, a, b, c, d, e, f, g, h, i, j):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1] + g*pos[:, 0]**3 + h*pos[:, 1]**3 + i*pos[:, 0]**2*pos[:, 1] + j * pos[:, 0]*pos[:, 1]**2


def myParametricModel_order1(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    pos = modules[1].manifold.gd[0].detach().view(-1, 2)

    modules[1]._ImplicitModule1__C = pol_order_1(pos, a, b, c).transpose(0, 1).unsqueeze(2)

def myParametricModel_order2(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    pos = modules[1].manifold.gd[0].detach().view(-1, 2)

    modules[1]._ImplicitModule1__C = pol_order_2(pos, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)
    alpha = 100.
    lam = 10.
    return lam*torch.norm(a**2 + b**2 + alpha*(c**2 + d**2 + e**2 + f**2))

def myParametricModel_order3(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    g = abc[6].unsqueeze(1)
    h = abc[7].unsqueeze(1)
    i = abc[8].unsqueeze(1)
    j = abc[9].unsqueeze(1)    
    pos = modules[1].manifold.gd[0].detach().view(-1, 2)

    modules[1]._ImplicitModule1__C = pol_order_3(pos, a, b, c, d, e, f, g, h, i ,j).transpose(0, 1).unsqueeze(2)

In [ ]:
aabb_ellipse = dm.Utilities.AABB.build_from_points(template)

density = 2
sigma = 3./math.sqrt(density)

pts_implicit1 = dm.Utilities.fill_area_uniform_density(dm.Utilities.area_shape, aabb_ellipse, density, shape=template, side=1)
print(pts_implicit1.shape)
angles = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(a) for a in angles])

In [ ]:
C = torch.ones(pts_implicit1.shape[0], 2, 1)
C.requires_grad_()
implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C, sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))

abc = torch.zeros(6, 2)
abc[0] = torch.ones(2)
abc.requires_grad_()

In [ ]:
my_atlas = dm.Models.Atlas(template, [implicit1, dm.DeformationModules.GlobalTranslation(2)], [dm.Attachment.VarifoldAttachement([0.1, 1., 2.5, 5., 10.])], len(peanuts), lam=10., model_precompute_callback=myParametricModel_order2, other_parameters=[abc], optimise_template=False)

In [ ]:
shoot_it = 10
shoot_method = 'euler'

In [ ]:
fitter = dm.Models.ModelFittingScipy(my_atlas, 1.)

costs = fitter.fit(peanuts, 100, options={'shoot_it': shoot_it, 'shoot_method': shoot_method}, log_interval=1)

In [ ]:
print(my_atlas.parameters[-1])

In [ ]:
plt.plot(range(len(costs)), costs)
plt.show()

In [ ]:
C_gt = torch.ones(pts_implicit1.shape[0], 2, 1)
C_gt[:, 0 , 0] = 1.*(pts_implicit1[:, 0]+torch.min(pts_implicit1[:, 0]))

In [ ]:
#template = my_atlas.compute_template()

%matplotlib qt5
ax = plt.subplot()
plt.plot(template[:, 0].numpy(), template[:, 1].numpy())
dm.Utilities.plot_C_arrow(ax, pts_implicit1, implicit1.C.detach(), alpha=0.3, scale=0.05, color='blue', mutation_scale=5.)
dm.Utilities.plot_C_arrow(ax, pts_implicit1, C_gt, alpha=0.3, scale=0.3, color='red', mutation_scale=5.)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C_gt, c_index=1, alpha=0.1, scale=0.01, color='C2')
plt.axis('equal')
plt.show()

In [ ]:
%matplotlib qt5
it_per_snapshot = 1
snapshots = int(shoot_it/it_per_snapshot)

#ht = my_atlas.compute_template().detach().view(-1, 2)

N = 10

for i in range(N):
    modules = dm.DeformationModules.CompoundModule(my_atlas.models[i].modules)
    modules.manifold.fill(my_atlas.models[i].init_manifold, copy=True)
    h = dm.HamiltonianDynamic.Hamiltonian(modules)
    intermediate_states, _ = dm.HamiltonianDynamic.shoot(h, shoot_it, shoot_method, intermediates=True)

    for j in range(snapshots):
        pos = intermediate_states[it_per_snapshot*j].gd[0].view(-1, 2).numpy()
        #pos_impl1 = intermediate_states[it_per_snapshot*j].gd[1][0].view(-1, 2).numpy()
        plt.subplot(N, snapshots + 1, i*snapshots + j + i + 1)
        plt.plot(pos[:, 0], pos[:, 1])
        #plt.plot(pos_impl1[:, 0], pos_impl1[:, 1], '.')
        plt.axis("equal")
        plt.axis(dm.Utilities.AABB.build_from_points(peanuts[i]).get_list())


    plt.subplot(N, snapshots + 1, i*snapshots + snapshots + i + 1)
    plt.plot(peanuts[i].numpy()[:, 0], peanuts[i].numpy()[:, 1])
    plt.axis("equal")
    plt.axis(dm.Utilities.AABB.build_from_points(peanuts[i]).get_list())